In [10]:
import pandas as pd
import numpy as np
WDIR = '/data/'

In [11]:
business_json_path = WDIR + "yelp_academic_dataset_business.json"
checkin_json_path = WDIR + "yelp_academic_dataset_checkin.json"
tip_json_path = WDIR + "yelp_academic_dataset_tip.json"
reviews_json_path = WDIR + "yelp_academic_dataset_review.json"
attributes_csv_path = WDIR + 'yelp_business_attributes.csv'

In [3]:
# Read in businesses df
df_b = pd.read_json(business_json_path, lines=True)

In [4]:
# Maintain a relevant set of businesses
relevant_businesses = set(df_b[df_b['state'].isin(['VA', 'NC'])].business_id.unique())

# Utility function to read in large dataframes and filter on the fly
def filter(path, critera, size=1000):
    chunk_list = []
    reader = pd.read_json(path, lines=True, chunksize=size)
    
    for chunk in reader:
        chunk_filter = chunk[chunk.business_id.isin(critera)]
        chunk_list.append(chunk_filter)
    
    reader.close()
    return pd.concat(chunk_list)

In [ ]:
# Read and save reviews file
df_revs = filter(reviews_json_path, relevant_businesses, 1000)
df_concat.to_pickle(WDIR + 'dfs/reviews.pkl')

In [11]:
# Read and save checkin file
df_checkin = filter(checkin_json_path, relevant_businesses, 1000)
df_checkin.to_pickle(WDIR + 'dfs/checkin.pkl')

In [ ]:
# Read and save tip file
df_tips = filter(tip_json_path, relevant_businesses, 1000)
df_tips.to_pickle(WDIR + 'dfs/tips.pkl')

In [51]:
# Read and save business attribs
df_att = pd.read_csv(attributes_csv_path)
df_att.to_pickle(WDIR + 'dfs/att.pkl')

In [25]:
import pickle
with open(WDIR + 'dfs/reviews.pkl', 'rb') as pickle_file:
    df_revs = pickle.load(pickle_file)
    
with open(WDIR + 'dfs/checkin.pkl', 'rb') as pickle_file:
    df_checkin = pickle.load(pickle_file)
    
with open(WDIR + 'dfs/tips.pkl', 'rb') as pickle_file:
    df_tips = pickle.load(pickle_file)

IOError: [Errno 2] No such file or directory: 'dfs/reviews.pkl'

In [26]:
bus_rev_dict = {}
for x in relevant_businesses:
    bus_rev_dict[x] = []
for x,c in df_revs.iterrows():
    try:
        bus_rev_dict[c['business_id']].append((c['text'], c['useful']))
    except:
        pass

In [27]:
for x,c in df_tips.iterrows():
    try:
        bus_rev_dict[c['business_id']].append((c['text'], 0))
    except:
        pass

In [28]:
from textblob import TextBlob
import nltk

In [45]:
def findavgsentiment(textarr):
    totalSent = 0
    for text, useful in textarr:
        blob = TextBlob(text)
        totalSent += blob.sentiment.polarity
    return totalSent/len(textarr)

In [29]:
def findavgsentiment_useful(textarr):
    totalSent = 0
    for text, useful in textarr:
        blob = TextBlob(text)
        totalSent += blob.sentiment.polarity*(useful+1)
    return totalSent/len(textarr)

In [ ]:
bus_score_dict = {}
for x in relevant_businesses:
    bus_score_dict[x] = []
for item in bus_rev_dict.items():
    bus_score_dict[item[0]] = findavgsentiment_useful(item[1])
    

In [53]:
bus_score_dict

{'fJkXfvCiVGJsjrIrUuLJCQ': 0.26234487508453674,
 'FqVTaF56MSgqCCX3N6hgLQ': 0.37071906565656565,
 'sV6kKpzneWwQ8cY8gQ8bgA': 0.263858991033446,
 'R2ir667KUOdyhGJp9Q06dQ': 0.24828639331498806,
 '4GrFgFdt3fYdZsYbnyRlZg': 0.10212637741046832,
 'x5fgrhaq8R8qXhZFPnfXWw': 0.2849578781257856,
 'VKdVPVkMOotZ-J248lHReQ': 0.25297619047619047,
 'pCb0jYxksLR562vV7As1bQ': 0.2867355784277791,
 'GlQs6zeCN0dwCe7gMBXFsg': 0.3225956664862915,
 'jdft5jzXyceJZLRtVL38VA': 0.13932570912903125,
 'iqx8VckOhO7nict32SPjKg': 0.13101826306357292,
 'wUHPgIh-rJThxPvxHPlsjQ': 0.1637833522721307,
 '4y8LLc9tuxqKhDSeRDUTDA': 0.2587474829916635,
 'huTkwb90PQw4HqGkbhz74Q': 0.2993022486772487,
 'VXFycyyI3uEx6IOHkPLvSQ': 0.3499585578832374,
 'vi2UlcBHPlGMTbGWO1e2-w': 0.3531944444444444,
 'l9tFVg7r7QXuLnCywMQl7Q': 0.29586083920224054,
 'ic5TJB0pUeiQ53J2CRBMig': 0.15404577234934375,
 'esB5D7EbZ5mJkqwSBK1yOA': -0.00465277777777779,
 '1Dx2W-2GL9LaE9j2vD5lfw': 0.13834928075637923,
 'at63ej-ri33Vqx4333Rd3Q': 0.12056925047861677,
 

In [22]:
df_b['WeightedSentiment'] = df_b['business_id'].apply(lambda x : bus_score_dict[x])

NameError: name 'bus_score_dict' is not defined

In [15]:
# Aggregate checkins (still have to merge)
from datetime import datetime

In [16]:
df_checkin['date'] = df_checkin['date'].apply(lambda x : list(map(lambda y : datetime.strptime(y, "%Y-%m-%d %H:%M:%S"), x.split(", "))))

In [17]:
df_checkin['checkins'] = df_checkin['date'].apply(len)

In [18]:
df_checkin['interval'] = df_checkin['date'].apply(lambda x : (x[-1] - x[0]).days if (x[-1] != x[0]) else 0)

In [23]:
merged = df_b.merge(df_checkin, how='inner', on='business_id').drop(['date'], axis=1)

In [ ]:
# filter business 

In [16]:
df_b = df_b[df_b['state'].isin(['VA', 'NC'])]

In [43]:
# convert categories label to iterables and then encode
a = df_b['categories'].apply(lambda x : str(x).split(', '))

In [44]:
# Use this to encode categories
from sklearn.preprocessing import MultiLabelBinarizer
b = MultiLabelBinarizer()
b.fit_transform(a.values)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
# @Todo:
"""
--> EDA:
    random scatter plots / histograms
    
-- > Preprocessing
- Merge checkins df (checkins, and interval) on business id with bussinesses df (filtered)
- Add sentiment column to business 
- Encode categories column in business dataframe (might want to filter categories to top N to reduce dimensionality)
- Encode 'attributes column' in business dataframe 
- Vectorize reviews column (do not have to do for this checkpoint)

---> SMOTE data before running any models (accounts for classification with an imbalanced class distr.)

--> run some basic models and note down the metrics
"""

In [47]:
from imblearn.over_sampling import SMOTE

0         Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh...
1         Health & Medical, Fitness & Instruction, Yoga,...
2                          Pets, Pet Services, Pet Groomers
3         Hardware Stores, Home Services, Building Suppl...
4         Home Services, Plumbing, Electricians, Handyma...
                                ...                        
209388                    Japanese, Sushi Bars, Restaurants
209389    Department Stores, Food, Mobile Phones, Fashio...
209390    American (New), Food, Burgers, Restaurants, Fa...
209391                     Pet Services, Pet Training, Pets
209392    Tax Services, Professional Services, Accountan...
Name: categories, Length: 209393, dtype: object

In [5]:
r = pd.read_pickle('data/dfs/reviews.pkl')
r.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
29,oyjkEn5fAqcGNsBbqc8LTw,0,2016-04-11 16:42:13,0,01DNCgnW8K6xSyv4krYa1w,1,Just... not good. We want it to be - we have ...,0,XivuRnPq3D-K5cPiBTztIA
58,l4ByLNdIcsjgnglB1aXQQQ,0,2018-04-15 14:54:30,0,BnB8z3d1BiLyqdAwgQnejw,1,I can't tell you how angry I am right now. I ...,1,fn0wmD6MY2zmNUkOJJrjjA
64,UOFTGqJ95xQxeYndG7hbbw,1,2011-08-25 00:49:25,0,PvgCQa3FsA-FXYQsaEguIw,5,I was staying at the Hilton next door and got ...,0,LhZhxlOqTSuRY6r7HV6NVg
66,gfhWUixZ4MmEAC1DtTsodA,0,2017-06-06 21:04:58,0,2dYcxwdtulsUY8kj4oZezw,5,GREAT SERVICE!! The food was amazing and very ...,0,iAQFEArbXp1WUrseOFTLMA
78,l_kefVF1frmC0xRW2YkvUA,0,2018-01-21 04:58:46,0,MRPCm_BYEHl2T9oNZYnCbQ,2,"Pros: Fun atmosphere, great for people watchin...",2,oAFF_A02Ws0mwH-lkbzyMQ


In [31]:
r.groupby(by='business_id').text.apply(list).reset_index(name='revs')

,business_id,revs
0,--7zmmkVg-IMGaXbuVd0SQ,[I'm a tad reluctant to write a review as I ru...
1,--EX4rRznJrltyn-34Jz1w,[As I walked through Northlake Mall I was capt...
2,--KCl2FvVQpvjzmZSPyviA,[It took me about a month from when I moved ac...
3,--U98MNlDym2cLn36BBPgQ,[Let's start with the bad. REALLY slow. I dine...
4,--VMPfs4zfZJtQbqzJsNhg,"[How good can a root canal be? Well, they are..."
...,...,...
16214,zzTM2KyJkKomLDkl6rM4dQ,[Mike was awesome. We had a backup to our main...
16215,zzbvgC8Rg0P8-IP8LnvYjA,[Horrible horrible experience \nPoor customer ...
16216,zzm9aR0UHEEEYdsR_zKcuw,[I completely ruined my hair today attempting ...
16217,zzmIMvqiBJ_-wVKg_OnGpw,[I was traveling to town for business and stum...


In [48]:
t = r.groupby(by='business_id').text.apply(list).reset_index(name='revs')
t['revs'] = t['revs'].apply(lambda x : " ".join(n.lstrip() for n in x))

In [49]:
t['revs'].iloc[0]

'I\'m a tad reluctant to write a review as I run the risk of spoiling one of the things I love about the place, it has just the right amount of \'busy\'.  Staff are awesome, and care that you have a great experience.  Outside patio is a wonderful.  It\'s the dogs bollocks. Thank you setting yourself apart from your competitors and offering a wide variety of beers! I enjoyed the Lemon Haze Pale Ale and my husband loved the Lucille Red Ale.  Keep up the fantastic beers and wide variety! Have been a bunch of times and can say they have great beer and great staff, especially the owners that make you feel like family. Awesome local hangout with really good beer. Happy to have them so close!! The fella and I stopped by with some friends on a Saturday afternoon to check it out before dinner. I agree with the other reviewer who said it feels like a cozy ski lodge. There are maybe 8-10 seats at the bar, two 4-top tables, and the rest of the seating is comfy-looking leather chairs scattered arou

In [20]:
r.groupby(by='business_id').useful.apply(np.sum).reset_index(name='useful')

,business_id,useful
0,--7zmmkVg-IMGaXbuVd0SQ,92
1,--EX4rRznJrltyn-34Jz1w,5
2,--KCl2FvVQpvjzmZSPyviA,15
3,--U98MNlDym2cLn36BBPgQ,4
4,--VMPfs4zfZJtQbqzJsNhg,61
...,...,...
16214,zzTM2KyJkKomLDkl6rM4dQ,13
16215,zzbvgC8Rg0P8-IP8LnvYjA,1
16216,zzm9aR0UHEEEYdsR_zKcuw,0
16217,zzmIMvqiBJ_-wVKg_OnGpw,134


In [21]:
r.groupby(by='business_id').text.apply(list).apply(len)

business_id
--7zmmkVg-IMGaXbuVd0SQ     68
--EX4rRznJrltyn-34Jz1w      5
--KCl2FvVQpvjzmZSPyviA     15
--U98MNlDym2cLn36BBPgQ      3
--VMPfs4zfZJtQbqzJsNhg     10
                         ... 
zzTM2KyJkKomLDkl6rM4dQ      9
zzbvgC8Rg0P8-IP8LnvYjA      6
zzm9aR0UHEEEYdsR_zKcuw      3
zzmIMvqiBJ_-wVKg_OnGpw    141
zztHqd_3ULxuIS8lkhYdwg      6
Name: text, Length: 16219, dtype: int64

In [26]:
pd.read_pickle(WDIR + 'dfs/tips.pkl')

,business_id,compliment_count,date,text,user_id
2,rDoT-MgxGRiYqCmi0bG10g,0,2016-07-18 22:03:42,Very nice good service good food,AY-laIws3S7YXNl_f_D6rQ
5,ALwAlxItASeEs2vYAeLXHA,0,2015-05-20 20:17:38,Great ramen! Not only is the presentation gorg...,HHNBqfbDR8b1iq-QGxu8ww
9,hfBrethLHS9iXeBNR8MtzQ,0,2011-09-30 18:38:47,Check out the great assortment of organic & co...,DsWg3leomfasGs3j0rOfbQ
12,gG9z6zr_49LocyCTvSFg0w,0,2014-06-14 00:44:00,Just be ready to stand in line. But it moves a...,oLHQ-31LcRobRxjRU6PrZA
25,a4grXFNKZmZm0ZJXd1PVdQ,0,2012-01-01 02:33:27,Great views of the city!,exEElE4pG-ZwoCMwoYddeg
...,...,...,...,...,...
1320663,EzUZU_y0ooTGbboxFck1Pg,0,2018-03-11 18:07:49,"Awesome pizza, and some of the best wings I ev...",mjkFsb-Cl3YIOalTk_15eQ
1320689,Gsbt1IoWkjrh9nExM7BIFw,0,2011-03-04 13:52:20,Get your Charlotte and North Carolina mugs her...,KI9-tjEmi24-A8PB2g1aPQ
1320701,RFN2um_vsoliBtvLzH7PcA,0,2012-08-28 23:27:33,"Few wings (chipotle honey are awesome), little...",dNm5OKmN0wYEB04qaHMsTA
1320721,VbY4yD6UmA9jKKxHscrxvg,0,2019-06-17 03:18:20,Delicious food. Main complaint? Warm sashimi,E1vswMSSB6wKBptbd_QRag


In [52]:
df_b['categories'] = df_b['categories'].apply(lambda x : x if x else '')

In [56]:
bc = ','.join(df_b['categories'])
cats = pd.DataFrame(bc.split(','), columns=['category'])
cats_ser = cats.category.value_counts()

In [57]:
cats_ser

 Restaurants          44488
 Shopping             26128
 Food                 24452
Restaurants           19456
 Home Services        16205
                      ...  
Wallpapering              1
Customs Brokers           1
Surf Schools              1
Dive Shops                1
 General Festivals        1
Name: category, Length: 2529, dtype: int64

In [13]:
import etl
import eda
from preprocess import TextProcessor

In [ ]:
# Extract data
etl_worker = ETL(etl.DATA_DIR + business_json_path, etl.DATA_DIR + checkin_json_path, etl.DATA_DIR + reviews_json_path, ['VA', 'NC'])

In [ ]:
# Format data
tp = TextProcessor(etl_worker.df)
tp.update_feats()
df = tp.df

In [ ]:
# EDA
eda.show_all(df)

In [ ]:
"""
 - Create pipeline for preprocessing and encoding
 - Embed or encode text features
 - SMOTE then TTS
 - Run models and tune
"""